In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.datasets as datasets
import imageio
import numpy as np
import matplotlib

from torchvision.utils import make_grid, save_image
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from tqdm import tqdm
from data.data import load_mnist

matplotlib.style.use('ggplot')

# learning parameters
batch_size = 512
epochs = 200
sample_size = 64 # fixed sample size
nz = 128 # latent vector size
k = 1 # number of steps to apply to the discriminator
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# MNIST Dataset
(xtrain, ytrain), (xval, yval), num_cls = load_mnist(final=True)

transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,),(0.5,)),
])

to_pil_image = transforms.ToPILImage()
train_data2 = datasets.MNIST(
    root='input/data',
    train=True,
    download=True,
    transform=transform
)

train_data = datasets.FashionMNIST(
    root='input/data',
    train=True,
    download=True,
    transform=transform
)
train_loader = DataLoader(train_data2, batch_size=batch_size, shuffle=True)

In [3]:
class Generator(nn.Module):
    def __init__(self, nz):
        super(Generator, self).__init__()
        self.nz = nz
        self.main = nn.Sequential(
            nn.Linear(self.nz, 256),
            nn.LeakyReLU(0.2),

            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),

            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),

            nn.Linear(1024, 784),
            nn.Tanh(),
        )

    def forward(self, x):
        return self.main(x).view(-1, 1, 28, 28)

In [4]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.n_input = 784
        self.main = nn.Sequential(
            nn.Linear(self.n_input, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),

            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),

            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),

            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = x.view(-1, 784)
        return self.main(x)

In [5]:
# to create real labels (1s)
def label_real(size):
    data = torch.ones(size, 1)
    return data.to(device)

In [6]:
# to create fake labels (0s)
def label_fake(size):
    data = torch.zeros(size, 1)
    return data.to(device)

In [7]:

# function to create the noise vector
def create_noise(sample_size, nz):
    return torch.randn(sample_size, nz).to(device)

In [8]:
# function to train the discriminator network
def train_discriminator(optimizer, data_real, data_fake):
    b_size = data_real.size(0)
    real_label = label_real(b_size)
    fake_label = label_fake(b_size)

    optimizer.zero_grad()

    output_real = discriminator(data_real)
    loss_real = criterion(output_real, real_label)

    output_fake = discriminator(data_fake)
    loss_fake = criterion(output_fake, fake_label)


    loss_real.backward()
    loss_fake.backward()
    optimizer.step()

    return loss_real + loss_fake

In [9]:
# function to train the generator network
def train_generator(optimizer, data_fake):
    b_size = data_fake.size(0)
    real_label = label_real(b_size)

    optimizer.zero_grad()

    output = discriminator(data_fake)
    loss = criterion(output, real_label)

    loss.backward()
    optimizer.step()

    return loss   

In [10]:
generator = Generator(nz).to(device)
discriminator = Discriminator().to(device)

print('##### GENERATOR #####')
print(generator)
print('######################')

print('\n##### DISCRIMINATOR #####')
print(discriminator)
print('######################')

# optimizers
optim_g = optim.Adam(generator.parameters(), lr=0.0002)
optim_d = optim.Adam(discriminator.parameters(), lr=0.0002)

# loss function
criterion = nn.BCELoss()

losses_g = [] # to store generator loss after each epoch
losses_d = [] # to store discriminator loss after each epoch
images = [] # to store images generatd by the generator

def train():
    # create the noise vector
    noise = create_noise(sample_size, nz)

    generator.train()
    discriminator.train()

    for epoch in range(epochs):
        loss_g = 0.0
        loss_d = 0.0
        for bi, data in tqdm(enumerate(train_loader), total=int(len(train_data)/train_loader.batch_size)):
            image, _ = data
            image = image.to(device)
            b_size = len(image)
            # run the discriminator for k number of steps
            for step in range(k):
                data_fake = generator(create_noise(b_size, nz)).detach()
                data_real = image
                # train the discriminator network
                loss_d += train_discriminator(optim_d, data_real, data_fake)
            data_fake = generator(create_noise(b_size, nz))
            # train the generator network
            loss_g += train_generator(optim_g, data_fake)

        # create the final fake image for the epoch
        generated_img = generator(noise).cpu().detach()
        # make the images as grid
        generated_img = make_grid(generated_img)
        images.append(generated_img)
        epoch_loss_g = loss_g / bi # total generator loss for the epoch
        epoch_loss_d = loss_d / bi # total discriminator loss for the epoch
        losses_g.append(epoch_loss_g)
        losses_d.append(epoch_loss_d)

        #SAVE IMAGE
        imgs = [np.array(to_pil_image(img)) for img in images]
        from PIL import Image
        im = Image.fromarray(imgs[epoch])
        im.save("Images_output/epoch_"+str(epoch)+".jpeg")

        
        print(f"Epoch {epoch} of {epochs}")
        print(f"Generator loss: {epoch_loss_g:.8f}, Discriminator loss: {epoch_loss_d:.8f}")

    return images

##### GENERATOR #####
Generator(
  (main): Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=0.2)
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): LeakyReLU(negative_slope=0.2)
    (6): Linear(in_features=1024, out_features=784, bias=True)
    (7): Tanh()
  )
)
######################

##### DISCRIMINATOR #####
Discriminator(
  (main): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): LeakyReLU(negative_slope=0.2)
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): LeakyReLU(negative_slope=0.2)
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=256, out_features=1

In [11]:
images = train()

118it [00:51,  2.31it/s]
  0%|          | 0/117 [00:00<?, ?it/s]

Epoch 0 of 200
Generator loss: 1.43534052, Discriminator loss: 0.92931348


118it [00:46,  2.53it/s]
  0%|          | 0/117 [00:00<?, ?it/s]

Epoch 1 of 200
Generator loss: 3.31022716, Discriminator loss: 1.16086638


118it [00:46,  2.54it/s]
  0%|          | 0/117 [00:00<?, ?it/s]

...

Epoch 197 of 200
Generator loss: 1.33440661, Discriminator loss: 1.06053555


118it [00:48,  2.42it/s]
  0%|          | 0/117 [00:00<?, ?it/s]

Epoch 198 of 200
Generator loss: 1.32612157, Discriminator loss: 1.06506228


118it [00:49,  2.41it/s]


Epoch 199 of 200
Generator loss: 1.29023063, Discriminator loss: 1.06532109


In [12]:
imgs = [np.array(to_pil_image(img)) for img in images]

imageio.mimsave('Images_output/generator_images.gif', imgs)
from PIL import Image
im = Image.fromarray(imgs[0])
im.save("Images_output/epoch_1.jpeg")